# Preprocessing

In [1]:
import pandas as pd
import numpy as np

In [2]:
work_dir = './data/kpatel28/'
using_year = '2016'
data_url = 'http://nflsavant.com/pbp_data.php?year='

* deal with missing values
    * Split the raw data into multiple tables
    * Explicitly fill irrelvant cases
    * Inpute data where possible
* Class consolidation for certain infrequent classes where appropriate
* assert statements to check dtype integrity and for missing values

In [3]:
file_path = work_dir + using_year + '-nfl-all.csv'
raw = pd.read_csv(file_path, sep=',')
raw.head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2016090800,2016-09-08,1,0,0,NaN,CAR,0,0,0,...,0,0,NaN,0,OWN,0,NaN,0,NaN,0
1,2016090800,2016-09-08,4,2,0,NaN,CAR,0,0,0,...,0,0,NaN,0,OWN,0,NaN,0,NaN,0
2,2016091100,2016-09-11,4,6,18,NaN,TB,0,0,0,...,0,0,NaN,0,OWN,0,NaN,0,NaN,0
3,2016091100,2016-09-11,4,0,0,NaN,TB,0,0,0,...,0,0,NaN,0,OWN,0,NaN,0,NaN,0
4,2016091101,2016-09-11,2,0,0,NaN,BUF,0,0,0,...,0,0,NaN,0,OWN,0,NaN,0,NaN,0


## Dealing with missing data

Here we drop all empty columns in the raw dataframe, define a function that will remove columns that contain unchanging (useless) columns, and then run the function on the raw data:

In [4]:
raw.dropna(axis=1, how='all', inplace=True) #Drop fully empty columns
def dropUnchanging(df): #Drops columns with values that never change
    to_drop = []
    for col in df:
        #print(col, ':', len(df[col].unique()))
        if (len(df[col].unique()) == 1):
            to_drop.append(col)
    print('\n' +str(len(to_drop)), 'columns with unchanging data will be dropped:', ', '.join(to_drop))
    return df.drop(to_drop, axis=1) #remove columns with unchanging values

In [5]:
raw = dropUnchanging(raw)


4 columns with unchanging data will be dropped: NextScore, TeamWin, SeasonYear, IsMeasurement


Now we'll check if there any columns with missing data. Indeed there are:

In [6]:
print('columns with missing data:')
missing = len(raw) - raw.count()
missing[missing > 0]

columns with missing data:


OffenseTeam       3376
Description          1
Formation          778
PlayType          1609
PassType         26632
RushDirection    33524
PenaltyTeam      42058
PenaltyType      42057
dtype: int64

In [7]:
assert(len(missing[missing > 0]) > 0)

### Penalties Table

PenaltyTeam and PenaltyType actually have little to no missing values. Both of these columns simply have empty data because very few of the rows record penalty events. A missing value in these columns means there wasn't a penalty. As we can see in the groupby, PenaltyTeam and PenaltyType have negligible missing values when they are relevant fields:

In [8]:
raw.groupby('IsPenalty')['IsPenalty', 'PenaltyTeam', 'PenaltyType'].count()

,IsPenalty,PenaltyTeam,PenaltyType
IsPenalty,,,
0,42056,0,0
1,3895,3893,3894


Penalty events are different enough from the rest of the data that they deserve their own dataframe:

In [9]:
penalty = raw[raw['IsPenalty']==1]
penalty = dropUnchanging(penalty)
missing = len(penalty) - penalty.count()
missing[missing > 0]
#penalty.info()
#penalty[penalty['PenaltyType'].isnull()]['Description'].iloc[0]


2 columns with unchanging data will be dropped: SeriesFirstDown, IsPenalty


PassType         2251
RushDirection    3331
PenaltyTeam         2
PenaltyType         1
dtype: int64

We save the new table:

In [10]:
file_path = work_dir + using_year + '-nfl-penalties.csv'
penalty.to_csv(file_path, index=False)

 We also drop all the penalty data from the original dataframe:

In [11]:
raw = raw.drop(raw[raw['IsPenalty']==1].index)
raw = raw.drop(['IsPenalty', 'PenaltyTeam', 'PenaltyType'], axis=1)
raw = dropUnchanging(raw)
missing = len(raw) - raw.count()
missing[missing > 0]


3 columns with unchanging data will be dropped: IsPenaltyAccepted, IsNoPlay, PenaltyYards


OffenseTeam       3376
Description          1
Formation          778
PlayType          1609
PassType         24381
RushDirection    30193
dtype: int64

### Other Events Table

"Zeroth Down" doesn't make much sense. Looking at cases when Down is zero shows that this case signifies events like timeout, end of a quarter, or the end of a game:

In [12]:
raw[raw['Down']==0][['GameId', 'Description', 'PlayType', 'PassType']].head(10)

,GameId,Description,PlayType,PassType
0,2016090800,END QUARTER 1,NaN,NaN
1,2016090800,TWO-MINUTE WARNING,NaN,NaN
2,2016091100,TIMEOUT #1 BY TB AT 06:18.,TIMEOUT,NaN
3,2016091100,END GAME,NaN,NaN
4,2016091101,END QUARTER 2,NaN,NaN
5,2016091102,END QUARTER 1,NaN,NaN
6,2016091102,"8-N.NOVAK EXTRA POINT IS GOOD, CENTER-46-J.WEE...",EXTRA POINT,NaN
7,2016091102,TIMEOUT #2 BY CHI AT 00:30.,TIMEOUT,NaN
9,2016091102,TIMEOUT #2 BY HOU AT 04:06.,TIMEOUT,NaN
10,2016091102,END GAME,NaN,NaN


This is good because based on the following groupby of down, we see that OffenseTeam, Formation, and PlayType have little to no missing data where it is most important (Downs 1-4):

In [13]:
raw.groupby('Down')['Down', 'OffenseTeam', 'Formation', 'PlayType', 'PassType', 'RushDirection'].count()

,Down,OffenseTeam,Formation,PlayType,PassType,RushDirection
Down,,,,,,
0,7167,3791,6389,5596,0,0
1,14365,14365,14365,14345,6770,6672
2,10624,10624,10624,10609,5884,4048
3,6480,6480,6480,6478,4739,1009
4,3420,3420,3420,3419,282,134


Now we'll segment out the "zeroth down" case into its own table called "other", save it, and then drop this data from the original (Note that kickoffs, one point conversions, and two point conversions will also be in this table):

In [14]:
other = raw[raw['Down']==0]
other = dropUnchanging(other)
missing = len(other) - other.count()
missing[missing > 0]


9 columns with unchanging data will be dropped: Down, ToGo, SeriesFirstDown, Yards, IsRush, IsIncomplete, PassType, IsSack, RushDirection


OffenseTeam    3376
Description       1
Formation       778
PlayType       1571
dtype: int64

According to the following event Description value_counts, the PlayType and Formation variables have missing values at events where they are not relevant (ie time outs or quarter endings). We'll fill these cases with strings to make them more explicit:

In [15]:
other[other['Formation'].isnull()]['Description'].value_counts()

END QUARTER 3    256
END QUARTER 2    255
END QUARTER 1    253
END QUARTER 4     13
Name: Description, dtype: int64

In [16]:
other['Formation'].fillna(value='NOT APPLICABLE', axis=0, inplace=True)

In [17]:
other[other['PlayType'].isnull()]['Description'].value_counts()

TWO-MINUTE WARNING    491
END QUARTER 3         256
END QUARTER 2         255
END QUARTER 1         253
END GAME              252
END QUARTER 4          13
TIMEOUT AT 12:11.       2
TIMEOUT AT 04:00.       2
TIMEOUT AT 13:49.       1
TIMEOUT AT 08:16.       1
TIMEOUT AT 13:45.       1
TIMEOUT AT 05:37.       1
TIMEOUT AT 10:46.       1
TIMEOUT AT 04:29.       1
TIMEOUT AT 11:11.       1
TIMEOUT AT 12:12.       1
TIMEOUT AT 07:18.       1
TIMEOUT AT 06:29.       1
TIMEOUT AT 14:12.       1
TIMEOUT AT 10:09.       1
TIMEOUT AT 04:10.       1
TIMEOUT AT 06:11.       1
TIMEOUT AT 05:18.       1
TIMEOUT AT 06:00.       1
TIMEOUT AT 05:51.       1
TIMEOUT AT 05:09.       1
TIMEOUT AT 08:34.       1
TIMEOUT AT 10:01.       1
TIMEOUT AT 11:46.       1
TIMEOUT AT 03:59.       1
TIMEOUT AT 12:19.       1
TIMEOUT AT 03:28.       1
TIMEOUT AT 06:40.       1
TIMEOUT AT 01:59.       1
TIMEOUT AT 06:37.       1
TIMEOUT AT 09:23.       1
TIMEOUT AT 03:33.       1
TIMEOUT AT 08:28.       1
TIMEOUT AT 0

In [18]:
other['PlayType'].fillna(value='NOT APPLICABLE', axis=0, inplace=True)

In [19]:
assert(other['Formation'].dtype == np.dtype('O'))
assert(other['PlayType'].dtype == np.dtype('O'))

The missing data in the OffenseTeam column is not an issue because, they don't occur where an offense/defense distinction is relevant. However, we will still keep the team name in 'Defense Team' in these cases because it indicates the relevant team in cases such as timeout:

In [20]:
other[other['OffenseTeam'].isnull()]['PlayType'].value_counts()

TIMEOUT           1772
NOT APPLICABLE    1570
EXCEPTION           33
PASS                 1
Name: PlayType, dtype: int64

Now lets fill the missing values for the 'Offense Team' column:

In [21]:
other['OffenseTeam'].fillna(value='NOT APPLICABLE', axis=0, inplace=True)

In [22]:
assert(other['OffenseTeam'].dtype == np.dtype('O'))

The Description column won't be directly used in visualization or to produce a machine learning feature, so we can just set the single missing value to 'UNKNOWN':

In [23]:
other.loc[other['Description'].isnull(), 'Description'] = 'UNKNOWN'

In [24]:
assert(other['Description'].dtype == np.dtype('O'))

In [25]:
assert((sorted(other['IsPass'].unique()) == np.array([0, 1])).all())
assert((sorted(other['IsTouchdown'].unique()) == np.array([0, 1])).all())
assert((sorted(other['IsChallenge'].unique()) == np.array([0, 1])).all())
assert((sorted(other['IsChallengeReversed'].unique()) == np.array([0, 1])).all())
assert((sorted(other['IsInterception'].unique()) == np.array([0, 1])).all())
assert((sorted(other['IsFumble'].unique()) == np.array([0, 1])).all())
assert((sorted(other['IsTwoPointConversion'].unique()) == np.array([0, 1])).all())
assert((sorted(other['IsTwoPointConversionSuccessful'].unique()) == np.array([0, 1])).all())
missing = len(other) - other.count()
assert(len(missing[missing > 0])==0)

In [26]:
file_path = work_dir + using_year + '-nfl-other.csv'
other.to_csv(file_path, index=False)

In [27]:
raw = raw.drop(raw[raw['Down']==0].index)
raw = dropUnchanging(raw)
missing = len(raw) - raw.count()
missing[missing > 0]


2 columns with unchanging data will be dropped: IsTwoPointConversion, IsTwoPointConversionSuccessful


PlayType            38
PassType         17214
RushDirection    23026
dtype: int64

### Plays Table

Now there are only three columns with missing data. PassType is only relevant during a pass and rush direction is only relevant during a rush. As such, there is no real missing data from either of these columns according to the following groupby: 

In [28]:
raw.groupby('PlayType')['PlayType', 'PassType', 'RushDirection'].count()

,PlayType,PassType,RushDirection
PlayType,,,
CLOCK STOP,68,0,0
EXCEPTION,63,0,0
FIELD GOAL,991,0,0
FUMBLES,92,0,0
PASS,17675,17675,0
PUNT,2060,0,0
QB KNEEL,373,0,0
RUSH,11863,0,11863
SACK,1064,0,0


Let's fill the missing data in PassType and RushDirection with 'NOT APPLICABLE' to make the distinction clear:

In [29]:
raw['PassType'].fillna(value='NOT APPLICABLE', axis=0, inplace=True)
raw['RushDirection'].fillna(value='NOT APPLICABLE', axis=0, inplace=True)

In [30]:
assert(raw['PassType'].dtype == np.dtype('O'))
assert(raw['RushDirection'].dtype == np.dtype('O'))

Finally, the small amount of missing data from PlayType can be filled with data from the Description column:

In [31]:
raw['PlayType'].value_counts()
noplay = raw[raw['PlayType'].isnull()]
noplay[['PlayType','Description','Formation']]

,PlayType,Description,Formation
1914,NaN,(8:13) DIRECT SNAP TO 21-J.MCKINNON. 48-Z.LIN...,UNDER CENTER
7365,NaN,(13:22) (SHOTGUN) DIRECT SNAP TO 25-L.MCCOY. ...,SHOTGUN
10098,NaN,(4:15) (SHOTGUN) DIRECT SNAP TO 86-J.WRIGHT. ...,SHOTGUN
10191,NaN,(14:50) DIRECT SNAP TO 29-D.JOHNSON. 29-D.JOH...,UNDER CENTER
15466,NaN,(9:43) (SHOTGUN) DIRECT SNAP TO 11-T.PRYOR. 1...,SHOTGUN
18171,NaN,(9:54) DIRECT SNAP TO 35-M.GILLISLEE. 35-M.GI...,UNDER CENTER
18177,NaN,(6:18) DIRECT SNAP TO 25-L.MCCOY. 25-L.MCCOY ...,UNDER CENTER
18221,NaN,(1:57) DIRECT SNAP TO 25-L.MCCOY. 25-L.MCCOY ...,UNDER CENTER
18271,NaN,(15:00) DIRECT SNAP TO 11-J.EDELMAN. 29-L.BLO...,UNDER CENTER
18302,NaN,(2:41) DIRECT SNAP TO 25-L.MCCOY. 25-L.MCCOY ...,UNDER CENTER


From the above view we can see that all the missing values are direct snap plays, lets fill these missing values:

In [32]:
raw['PlayType'].fillna(value='DIRECT SNAP', axis=0, inplace=True)

In [33]:
assert(raw['PlayType'].dtype == np.dtype('O'))

In [34]:
assert((sorted(raw['IsRush'].unique()) == np.array([0, 1])).all())
assert((sorted(raw['IsPass'].unique()) == np.array([0, 1])).all())
assert((sorted(raw['IsIncomplete'].unique()) == np.array([0, 1])).all())
assert((sorted(raw['IsTouchdown'].unique()) == np.array([0, 1])).all())
assert((sorted(raw['IsSack'].unique()) == np.array([0, 1])).all())
assert((sorted(raw['IsChallenge'].unique()) == np.array([0, 1])).all())
assert((sorted(raw['IsChallengeReversed'].unique()) == np.array([0, 1])).all())
assert((sorted(raw['IsInterception'].unique()) == np.array([0, 1])).all())
assert((sorted(raw['IsFumble'].unique()) == np.array([0, 1])).all())
missing = len(raw) - raw.count()
assert(len(missing[missing > 0])==0)

Now that all of the notable missing data has been accounted for, lets save the table that contains most of the game play. This is the most important table and where most of the analysis and modelling will come from:

In [35]:
file_path = work_dir + using_year + '-nfl-plays.csv'
raw.to_csv(file_path, index=False)

## Data Imputation and Consolidation

This section of the notebook will deal with small data filling/inputations and consolidating rare or one-off data values.

In [36]:
file_path = work_dir + using_year + '-nfl-plays.csv'
plays = pd.read_csv(file_path)

As is clear below, many of the possible values of PassType are meangingless. Even worse, they are one-off values that would be useless to a learning algorithm. We will consolidate all of these infrequent values into a new value - 'UNKNOWN':

In [37]:
pass_values = plays[plays['IsPass']==1]['PassType'].value_counts()
pass_values

SHORT RIGHT                   5891
SHORT LEFT                    5213
SHORT MIDDLE                  3404
DEEP RIGHT                    1223
DEEP LEFT                     1212
DEEP MIDDLE                    716
INTENDED FOR                     4
NOT LISTED                       3
BACK TO                          1
RIGHT (58-JHICKS)                1
(6:33) 11-ASMITH                 1
MIDDLE TO                        1
(55-A.BROOKS) [53-NBOWMAN]       1
(94-C.LIUGET) [99-JBOSA]         1
(:15) (SHOTGUN)                  1
(4:03) (NO                       1
KESSLER THROUGH                  1
Name: PassType, dtype: int64

In [38]:
plays.loc[plays['PassType'].isin(list(pass_values.iloc[6:].index)), 'PassType'] = 'UNKNOWN'

In [39]:
assert('NOT LISTED' not in plays['PassType'].unique())
assert(len(plays[plays['PassType']=='UNKNOWN']) <= 16)
assert(plays['PassType'].dtype == np.dtype('O'))

In [40]:
file_path = work_dir + using_year + '-nfl-plays.csv'
plays.to_csv(file_path, index=False)

The Formation value 'WILDCAT' is another rare value that can be consolidated. The Wildcat formation for football corresponds to the Direct Snap play. In order for the Formation column to be consistent with the PlayType column, the WILDCAT value must be converted to either SHOTGUN, UNDER CENTER, or NO HUDDLE SHOTGUN. Based on the Description column, all the WILDCAT values should be consolidated to SHOTGUN:

In [41]:
plays['Formation'].value_counts()

SHOTGUN              17064
UNDER CENTER         11177
NO HUDDLE SHOTGUN     3042
PUNT                  2046
FIELD GOAL             992
NO HUDDLE              565
WILDCAT                  3
Name: Formation, dtype: int64

In [42]:
plays[plays['Formation']=='WILDCAT']['Description']

18750    (12:39) (SHOTGUN) 11-T.AUSTIN RIGHT END TO NYG...
18761    (7:26) (SHOTGUN) 23-B.CUNNINGHAM LEFT TACKLE T...
27225    (15:00) 32-R.KELLEY RIGHT GUARD TO DAL 6 FOR -...
Name: Description, dtype: object

In [43]:
plays.loc[plays['Formation']=='WILDCAT', 'PlayType'] = 'DIRECT SNAP'
plays.loc[plays['Formation']=='WILDCAT', 'Formation'] = 'SHOTGUN'

In [44]:
assert('WILDCAT' not in plays['Formation'].unique())
assert(plays['PlayType'].dtype == np.dtype('O'))
assert(plays['Formation'].dtype == np.dtype('O'))

As a final change, let's flip the values of IsIncomplete and rename it to IsCompleted for future convenience:

In [45]:
old = plays['IsIncomplete'].copy()
plays['IsIncomplete'] = 1 - plays['IsIncomplete']
plays.rename(columns={'IsIncomplete': 'IsCompleted'}, inplace=True)

In [46]:
assert('IsIncomplete' not in plays.columns)
assert((old == 1 - plays['IsCompleted']).all())

In [47]:
assert((sorted(plays['IsRush'].unique()) == np.array([0, 1])).all())
assert((sorted(plays['IsPass'].unique()) == np.array([0, 1])).all())
assert((sorted(plays['IsCompleted'].unique()) == np.array([0, 1])).all())
assert((sorted(plays['IsTouchdown'].unique()) == np.array([0, 1])).all())
assert((sorted(plays['IsSack'].unique()) == np.array([0, 1])).all())
assert((sorted(plays['IsChallenge'].unique()) == np.array([0, 1])).all())
assert((sorted(plays['IsChallengeReversed'].unique()) == np.array([0, 1])).all())
assert((sorted(plays['IsInterception'].unique()) == np.array([0, 1])).all())
assert((sorted(plays['IsFumble'].unique()) == np.array([0, 1])).all())

In [48]:
file_path = work_dir + using_year + '-nfl-plays.csv'
plays.to_csv(file_path, index=False)